In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from collections import Counter

# Load the training dataset
dataset = pd.read_csv('creditcard.csv')

# Drop the "Time" column
dataset = dataset.drop('Time', axis=1)

# Split into input and output elements
X, y = dataset.values[:,:-1], dataset.values[:,-1]

# Print key information about the dataset
print(f"Dataset shape: {X.shape}")
print(f"Input features: {dataset.columns[:-1]}")
print(f"Target variable: {dataset.columns[-1]}")
print(f"Class distributions: {Counter(y)}")


Dataset shape: (284807, 29)
Input features: Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount'],
      dtype='object')
Target variable: Class
Class distributions: Counter({np.float64(0.0): 284315, np.float64(1.0): 492})


In [3]:

# Split into train and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [4]:

# Define parameter grid
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.05],
    'n_estimators': [100, 200, 300],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Compute the positive class weight
class_weight = (len(y_train) - np.sum(y_train)) / np.sum(y_train)

# Create XGBClassifier
model = XGBClassifier(objective='binary:logistic', scale_pos_weight=class_weight, tree_method='hist', random_state=42, n_jobs=2)

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc', cv=3, n_jobs=4, pre_dispatch=4)
grid_search.fit(X_train, y_train)

# Print best score and parameters
print(f"Best score: {grid_search.best_score_:.3f}")
print(f"Best parameters: {grid_search.best_params_}")


Best score: 0.983
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 200, 'subsample': 0.8}


In [5]:

# Access best model
best_model = grid_search.best_estimator_

# Save best model
best_model.save_model('best_model_creditcard.ubj')


In [6]:

# Load saved model
loaded_model = XGBClassifier()
loaded_model.load_model('best_model_creditcard.ubj')

# Use loaded model for predictions
predictions = loaded_model.predict(X_valid)


In [7]:

# Print roc auc score
rocauc = roc_auc_score(y_valid, predictions)
print(f"ROC AUC: {rocauc:.3f}")

ROC AUC: 0.934
